In [ ]:
import numpy as np
import cv2
import glob
import dlib
import math
import itertools
import pickle
import pandas as pd
import os
from math import sqrt

In [ ]:
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("/Users/guvenerkaya/Downloads/dlib_shape_predictor") 

data = {}

In [ ]:
def get_landmarks(image):
    data_points = []
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) #convert to grayscale
    clahe_image = clahe.apply(gray) #Contrast Limited Adaptive Histogram Equalization
    detections = detector(gray, 1)
    if len(detections) > 0:
        for k,d in enumerate(detections): #For all detected face instances individually
            shape = predictor(image, d) #Draw facial landmarks with dlib's predictor class
            xlist = []
            ylist = []
            for i in range(0,68): #Store X and Y coordinates of the facial landmarks in two lists
                xlist.append(float(shape.part(i).x))
                ylist.append(float(shape.part(i).y))
            landmarks = []
            for x, y in zip(xlist, ylist):
                landmarks.append(x) # get them in order in the same list wit(x_1,y_1,x_2,y_2...)
                landmarks.append(y)
            data['landmarks'] = landmarks 
            data_points.append(data['landmarks'])
        return data_points
    else:
        return None

In [ ]:
key_landmarks = []
no_face_detected = []
detected_faces = []
video_num = []
keyframe_num =[]

files = glob.glob('/Volumes/LIRIS_ACCEDE/keyframes 2/*.png') #read all extracted key frames
for file in files:
    
#image = cv2.imread('/Volumes/LIRIS_ACCEDE/keyframes 2/ACCEDE00195_03.png') # read image/frame
    image = cv2.imread(file) #store each key frame one by one to draw facial landmarks
    data_points = get_landmarks(image) #store facial landmarks 
    if data_points != None: #if face is found
        for i in range(len(data_points)): #adding the facial landmarks of each found face to the list key_landmarks
            key_landmarks.append(data_points[i])
            video_number = file[40:45] #detecting which video the key frames belong 
            video_num.append(video_number)
            keyframe_number = file[46:48]#detecting the which keyframe of the video it is
            keyframe_num.append(keyframe_number)
            detected_faces.append(i+1)#detecting the number of detected faces in the key frame
         
    #else:
      #  no_face_detected.append(file[34:])


#df1 = pd.DataFrame(data_points, columns = name)
#print(len(files))    
#print(files)
#set column names
name_x = np.arange(1, 69, 1)
name_x = list(map(str, name_x))
name_x = ["x_" + item for item in name_x]

name_y = np.arange(1, 69, 1)
name_y = list(map(str, name_y))
name_y = ["y_" + item for item in name_y]

name = [None]*(len(name_x)+len(name_y))
name[::2] = name_x
name[1::2] = name_y

df = pd.DataFrame(key_landmarks, index = video_num,   columns=name)
df.set_index( pd.Index(keyframe_num), append=True, inplace=True)
df.set_index(pd.Index(detected_faces), append = True, inplace= True)
df.index.names = ['id','keyframe','face']
df_original = df
df
#video_num
#keyframe_num
#pickle.dump(df, open('/Users/guvenerkaya/Desktop/datasets_and_models/kf_facial_landmarks', 'wb')) # save key frames with drawn facial landmarks




In [ ]:
df = df.values

final_data = []
#calculate the euclidean distances of facial landarks of the key frames
for row in range(0, len(df)):
    data = df[row]
    #print(data)
    print("\n")
    x = []
    y = []
    new_data = []
    x = data[::2]
    y = data[1::2]
    #print(x)
    #print("\n")
    #print(y)
    #print("\n")
    for i in range(0, 68):
        for j in range(i+1, 68):
            distance = sqrt((x[i] - x[j]) * (x[i] - x[j]) + (y[i] - y[j]) * (y[i] - y[j])) 
            new_data.append(distance)
    final_data.append(new_data)
#set column names
names = []
for i in range(1, 69, 1):
    for j in range(i+1, 69, 1):
        name = "distance" + str(i) + "<->" + str(j)
        names.append(name)

df_new = pd.DataFrame(final_data, columns=names, index=df_original.index)
#extend the data frame with euclidean distances
df3 = pd.concat([df_original, df_new], axis=1, sort=False)
df3

In [ ]:
#assess only the best features
best = ['distance64<->66', 'distance25<->44', 'distance45<->47', 'distance65<->68', 'distance44<->48', 'distance20<->39', 'distance63<->67', 'distance34<->63', 'distance62<->68', 'distance60<->62', 'distance21<->39', 'distance21<->41', 'distance52<->67', 'distance54<->56', 'distance54<->68', 'distance21<->40', 'distance49<->65', 'distance53<->57', 'distance20<->42', 'distance10<->55', 'distance36<->57', 'distance53<->66', 'distance49<->58', 'distance24<->43', 'distance57<->64', 'distance9<->61', 'distance64<->67', 'distance50<->68', 'distance49<->59', 'distance25<->47', 'distance32<->34', 'distance31<->63', 'distance39<->42', 'distance49<->60', 'distance59<->62', 'distance60<->61', 'distance51<->53', 'distance35<->55', 'distance38<->42', 'distance49<->56', 'distance50<->57', 'distance63<->65', 'distance31<->62', 'distance21<->42', 'distance32<->61', 'distance20<->41', 'distance54<->61', 'distance58<->63', 'distance10<->65', 'distance9<->59']
df4 = df3[best] 
df4 #saved as 'kf_50_best_distances'


In [ ]:
SVM = pickle.load(open('/Users/guvenerkaya/Desktop/datasets_and_models/SVM_trained', 'rb'))#load trained svm 
test  = SVM.predict_proba(df4)  #find the probabilities  
#test = SVM.predict(df5)
a =pd.DataFrame(test)
#a
x = df4.index #setindex
a.index = x
SVM.classes_

In [ ]:
a.columns = ['angry', 'contemptuous', 'disgusted', 'fearful' ,'happy', 'neutral' ,'sad', 'surprised']#set columns according to SVM classes
#by_row_index = a.groupby(a.index)
#df_means = by_row_index.mean(level='id')
df_means= a.mean(level='id')  #calculate the means of the emotion prabibilities with the same id
index = df_means.index.to_list 

index_int = df_means.index.astype(str).astype(int) #change type of index to manipulate it
df_means.index = index_int
df_means

In [ ]:
#find_missing finds the missing values in a given list of numbers
def find_missing(lst): 
    return [x for x in range(lst[0], lst[-1]+1)  
                               if x not in lst] 
feature_list=['angry', 'contemptuous', 'disgusted' ,'fearful','happy', 'neutral', 'sad',
 'surprised']
zero_data = np.zeros(shape=(4534,len(feature_list)))
feature_list=['angry', 'contemptuous', 'disgusted' ,'fearful','happy', 'neutral', 'sad',
 'surprised']
#make a zero dataframe for missing indices in key frame emotion probabilities (when no faces found in key frames)
index_list = find_missing(index_int)
df0 = pd.DataFrame(zero_data,index =index_list , columns=feature_list)
df0
last_df = df_means.append(df0,ignore_index=False)
last_df.index.name = 'id'
last_df

df_sorted = last_df.sort_values('id', axis=0) #sort the dataframe depending the index values
last_row = [0,0,0,0,0,0,0,0]
add_last_row =  pd.DataFrame({'angry':0, 'contemptuous':0, 'disgusted':0, 'fearful':0, 'happy':0, 'neutral':0, 'sad':0, 'surprised':0}, index =[0])
#last_of_us = pd.concat([add_last_row, df_sorted],)
#pickle.dump(last_of_us, open('/Users/guvenerkaya/Desktop/datasets_and_models/emotion_proabilities', 'wb'))
#index_list[-1]
last_of_us = pd.concat([add_last_row,df_sorted],  ignore_index=True)


In [ ]:
# set detailed index for emotion probabilities
lastlast= last_of_us.set_index(index_dfr.index)
index_dfr = pickle.load(open('/Users/guvenerkaya/Desktop/datasets_and_models/valence_dataframe','rb'))
#pickle.dump(last_of_us, open('/Users/guvenerkaya/Desktop/datasets_and_models/emotion_probabilities', 'wb'))
lastlast

In [ ]:
#add emotion probabilities to arousal dataframe
arousal_data = pd.read_pickle('/Users/guvenerkaya/Desktop/datasets_and_models/arousal_dataframe')
arousal_features = arousal_data.iloc[:,:-1]
arousal_new = arousal_data.reset_index(drop=True)
arousal_new.iloc[-1]
concencated = arousal_new.join(last_of_us)
final= concencated.drop(labels='arousalValue', axis = 1)
final.insert(31, 'arousalValue',arousal_new.iloc[:,-1])
final
#arousal dataframe with emotion probabilities are saved as 'arousal_with_emotionsprobabilities'
#index_df = pickle.load(open('/Users/guvenerkaya/Desktop/datasets_and_models/valence_dataframe','rb'))
finalfinal=final.set_index(index_df.index) # set index including the ids and videonames
#pickle.dump(finalfinal,open('arousal_with_emotionsprobabilities','wb'))

#add emotion probabilities to valence dataframe and save as 'valence_with_emotionsprobabilities'
valence_data= pd.read_pickle('/Users/guvenerkaya/Desktop/datasets_and_models/valence_dataframe')
valence_features = valence_data.iloc[:,:-1]
concat = valence_features.join(lastlast)
concat
concat.insert(25,'valenceValue',valence_data.iloc[:,-1])
concat
pickle.dump(concat,open('/Users/guvenerkaya/Desktop/datasets_and_models/valence_with_emotionsprobabilities','wb'))

In [ ]:
y= final.iloc[:,31]
X=final.iloc[:,:31]
X


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.24, random_state=0)
lr= LinearRegression()
lr.fit(X_train,y_train)

In [ ]:
y_pred = lr.predict(X_test)
from sklearn.metrics import r2_score
r2_score(y_test,y_pred)
import scipy
scipy.stats.pearsonr(y_test, y_pred)

In [ ]:
arousal_X = arousal_data.iloc[:,:23]
arousal_y = arousal_data.iloc[:,23]
X_train1, X_test1, y_train1, y_test1 = train_test_split(arousal_X, arousal_y, test_size=0.25, random_state=0)
lr_1 = LinearRegression()
lr_1.fit(X_train1,y_train1)
y_pred1 = lr_1.predict(X_test1)
r2_score(y_test1,y_pred1)